# Классификация комментариев с BERT и TF-IDF

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Цели:
* Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
* Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm import notebook

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import torch
import transformers
from transformers import AutoModel, AutoTokenizer 

from catboost import CatBoostClassifier, Pool, cv

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
nltk.download('popular', quiet=True);

In [3]:
pd.options.display.float_format = '{:0.3f}'.format
pd.options.mode.chained_assignment = None

%config InlineBackend.figure_format = 'retina'

Загрузка комментариев.

In [4]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
df.shape

(159571, 2)

Оценим сбалансированность выборки.

* Токсичных комментариев намного меньше.
* Присутствует дисбаланс классов.

In [5]:
df.toxic.value_counts() / df.toxic.size

0   0.898
1   0.102
Name: toxic, dtype: float64

Очистим текст при помощи регулярных выражений. Это впоследствии пригодится для tf-idf с CatBoost и поможет провести проверку на дубликаты. 

In [6]:
# Ограничим число символов - 800. Этого достаточно для задачи и поможет увеличить скорость расчетов.
def clear_text(text, char_limit=800):
    # Приведем текст к нижнему регистру очистив его от лишних символов и цифр.
    text = re.sub(r"[^a-z']", ' ', text.lower())
    
    # Обрежим сообщение до ближайшего целого слова.
    try:
        r = re.match(r'(?P<truncated> .+) \s', text[:char_limit], re.X)
        text = r['truncated'].split()
        return ' '.join(text)
    except:
        return 'empty line'

In [7]:
text = df['text'].map(clear_text).copy()
text.head()

0    explanation why the edits made under my userna...
1    d'aww he matches this background colour i'm se...
2    hey man i'm really not trying to edit war it's...
3    more i can't make any real suggestions on impr...
4    you sir are my hero any chance you remember wh...
Name: text, dtype: object

Теперь проверим дубликаты и удалим их.

In [8]:
text.duplicated().sum()

1570

In [9]:
df = df[~text.duplicated()]
text.drop_duplicates(inplace=True)
df.shape

(158001, 2)

Датафрейм для tf-idf с CatBoost.

In [10]:
text_df = pd.concat([text, df['toxic']], axis=1)

Обработаем текст при помощи лемматизатора Wordnet из NLTK.

In [11]:
# Инициализируем WordNetLemmatizer.
lemmatizer = WordNetLemmatizer()

# Функция для определения формы речи перед лемматизацией.
def get_wordnet_pos(word):
    # nltk.pos_tag() возвращает кортеж с тегом POS.
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Функция лемматизации.
def lemmatize(text):
    return ' '.join(
        [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    )

Лемматизация процесс долгий. Воспользуемся pickle для сохранения и загрузки её результатов.

In [12]:
try:
    with open('lemmatized_test.pickle', 'rb') as f:
        text = pickle.load(f)
except:
    text = text.map(lemmatize)
    with open('lemmatized_test.pickle', 'wb') as f:
        pickle.dump(text, f)

# Обучение

Проведем последовательное обучением различных моделей для получения более подробного отчета.

## CatBoostClassifier

Разделим выборку и сформируем Pool'ы для CatBoost.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    text_df['text'],
    text_df['toxic'],
    test_size=0.2,
    stratify=text_df['toxic'],
    random_state=38)

# Преобразуем в DataFrame чтобы можно было выбрать text_features в CatBoost Pool.
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
train_set = Pool(X_train, y_train, text_features=['text'])

Проведем кросс-валидацию.

In [14]:
# Гиперпараметры.
params = {
    'objective': 'Logloss',
    'loss_function': 'Logloss',
    'depth': 6,
    'subsample': 0.8,
    'random_strength': 0.3,
}

In [15]:
cv_data = cv(
    pool=train_set,
    params=params,
    early_stopping_rounds=100,
    logging_level='Silent',
    fold_count=4,
    stratified=True,
    partition_random_seed=38)

# Результаты кросс-валидации.
cv_data.iloc[-1, :]

iterations           999.000
test-Logloss-mean      0.120
test-Logloss-std       0.003
train-Logloss-mean     0.114
train-Logloss-std      0.001
Name: 999, dtype: float64

Проверим метрики на тестовой выборке.

In [16]:
cb = CatBoostClassifier(random_state=38, **params)
cb.fit(train_set, verbose=250)

print()
print(classification_report(y_test, cb.predict(X_test)))

Learning rate set to 0.081354
0:	learn: 0.5742707	total: 57.5ms	remaining: 57.4s
250:	learn: 0.1199002	total: 20.6s	remaining: 1m 1s
500:	learn: 0.1102371	total: 40.4s	remaining: 40.2s
750:	learn: 0.1030989	total: 1m	remaining: 20s
999:	learn: 0.0976653	total: 1m 19s	remaining: 0us

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     28388
           1       0.86      0.68      0.76      3213

    accuracy                           0.96     31601
   macro avg       0.91      0.83      0.87     31601
weighted avg       0.95      0.96      0.95     31601



## TF-IDF

In [17]:
# Подгрузим стоп слова.
nltk_stopwords = stopwords.words('english')
# Векторизация TF-IDF текстовых данных.
vectorizer = TfidfVectorizer(max_df=0.8, # дает лучше результат чем nltk_stopwords
                             max_features=6000)

X = vectorizer.fit_transform(text)
X.shape

(158001, 6000)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df.loc[text.index, 'toxic'],
    test_size=0.2,
    stratify=df.loc[text.index, 'toxic'],
    random_state=38)

print(X_train.shape, X_test.shape)

(126400, 6000) (31601, 6000)


Посчитаем основные метрики полученные у LogisticRegression.

In [19]:
lr = LogisticRegression(max_iter=500,
                        class_weight={0: 1, 1: 2},
                        n_jobs=-1,
                        C=5,
                        random_state=38)
lr.fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     28388
           1       0.80      0.75      0.78      3213

    accuracy                           0.96     31601
   macro avg       0.89      0.86      0.88     31601
weighted avg       0.95      0.96      0.96     31601



## BERT - unitary/toxic-bert model

Будем использовать AutoModel и AutoTokenizer из библиотеки transformers.

In [20]:
# Зададим модель, она как раз заточена под toxic comments.
model_name = 'unitary/toxic-bert'

# Загрузим предобученную PyTorch модель.
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Отредактируем настройки, в частности поменяем problem_type с мультиклассовой классификации.

In [21]:
model.config = model.config.from_dict(
    {
        'id2label': {'0': 'not_toxic', '1': 'toxic'},
        'label2id': {'not_toxic': '0', 'toxic': '1'},
        'problem_type': 'single_label_classification'
    }
)

Преобразуем данные. На выходе получим input tokens и маску значимости.

In [22]:
input_ids, _, attention_mask = tokenizer(df['text'].values.tolist(),
                                         return_tensors='pt',
                                         max_length=256, # ограничим длину, иначе всё очень долго считается.
                                         padding=True, 
                                         truncation=True).values()

Сформируем окончательные признаки.

In [23]:
# Попробуем загрузить фичи.
try:
    with open('features_full.pickle', 'rb') as f:
        features = pickle.load(f)    
except:
    # Количесто батчей чтобы не забить всю память расчетами.
    batch_size = 100
    size = df.shape[0]
    embeddings = []

    for i in notebook.tqdm(range(size // batch_size + (size % batch_size != 0))):
        with torch.no_grad():
            outputs = model(input_ids[batch_size*i:batch_size*(i+1)],
                            attention_mask=attention_mask[batch_size*i:batch_size*(i+1)])
            # Найдем среднее в выходном тензоре.
            outputs_mean = torch.mean(outputs[0], dim=1)
            embeddings.append(outputs_mean)

    with open('features_full.pickle', 'wb') as f:
        pickle.dump(features, f)
        
    features = np.concatenate(embeddings)

In [24]:
features.shape

(158001, 768)

Разбивка выборок.

In [25]:
X_train, X_test, y_train, y_test = \
    train_test_split(features, df['toxic'], test_size=0.2, random_state=38, stratify=df['toxic'])

Предсказание модели.

In [28]:
lr = LogisticRegression(n_jobs=-1)
lr.fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     28388
           1       0.94      0.95      0.94      3213

    accuracy                           0.99     31601
   macro avg       0.97      0.97      0.97     31601
weighted avg       0.99      0.99      0.99     31601



# Выводы

* Получена модель, классифицирующая комментарии на позитивные и негативные.
* Достигнута f1 ~ 0.94 на тестовой выборке благодаря правильно выбранной модели unitary/toxic-bert.
* Проведено сравнение BERT, CatBoostClassifier и TF-IDF.
    * Если доступно много ресурсов и нужна повышенная точность, то лучше всего использовать трасформеры.
    * Если требуется быстрый результат из коробки, то побеждает CatBoostClassifier.
* Векторизация и последующая классификация TF-IDF показала себя неплохо, но возможно следует выбрать другой классификатор для достижения более высоких результатов.